<a href="https://colab.research.google.com/github/lisaong/mldds-courseware/blob/master/video_classification_CNN_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Video Classification with RNN

This notebook demonstrates video classification of action videos using a time-distributed CNN + RNN.

The CNN will use the InceptionV3 model (transfer learning) to perform feature extraction.

1. Run on Colab
2. Runtime -> Change Runtime Type: GPU

We will be using a subset of techniques demonstrated in the more complex example here: https://github.com/harvitronix/five-video-classification-methods

We will also be using a small dataset to demonstrate how this works. Only 5 classes were selected from the UCF101 video classification dataset.

Incidentally, this entire dataset is also part of Tensorflow (but already preprocessed): https://www.tensorflow.org/datasets/catalog/ucf101


In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [2]:
# Ensure that ffmpeg is installed as we will be using it to extract frames
!apt install ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 7 not upgraded.


In [3]:
# Instead of 101 classes, we will start small, just 5 classes.

classes = [
  'Archery',
  'Basketball',
  'CricketBowling',
  'Diving',
  'Haircut'
]

classes

['Archery', 'Basketball', 'CricketBowling', 'Diving', 'Haircut']

In [4]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9681 sha256=89025d17a408b62f5a405f00e7aa2ca6ed957bb10ac95b081d482a56ddc38a2b
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [0]:
# Download subset of the dataset for the classes we have.
import wget
import zipfile
import os

def download_and_extract(label, folder):
  url = f'https://github.com/lisaong/mldds-courseware/raw/master/data/ucf101-5classes/{folder}/{label}.zip'
  print(f'Downloaded {url}')

  # prepare folders
  if not os.path.isdir(folder):
    os.mkdir(folder)
  filename = f'{label}.zip'

  # download
  wget.download(url, filename)

  # extract within folder
  with zipfile.ZipFile(filename) as f:
    f.extractall(folder)
  print(f'Extracted {os.path.join(folder, label)}')
  
  # delete zip archive
  os.remove(filename)

In [16]:
for label in classes:
  download_and_extract(label, 'train')
  download_and_extract(label, 'test')

Downloaded https://github.com/lisaong/mldds-courseware/raw/master/data/ucf101-5classes/train/Archery.zip
Extracted train/Archery
Downloaded https://github.com/lisaong/mldds-courseware/raw/master/data/ucf101-5classes/test/Archery.zip
Extracted test/Archery
Downloaded https://github.com/lisaong/mldds-courseware/raw/master/data/ucf101-5classes/train/Basketball.zip
Extracted train/Basketball
Downloaded https://github.com/lisaong/mldds-courseware/raw/master/data/ucf101-5classes/test/Basketball.zip
Extracted test/Basketball
Downloaded https://github.com/lisaong/mldds-courseware/raw/master/data/ucf101-5classes/train/CricketBowling.zip
Extracted train/CricketBowling
Downloaded https://github.com/lisaong/mldds-courseware/raw/master/data/ucf101-5classes/test/CricketBowling.zip
Extracted test/CricketBowling
Downloaded https://github.com/lisaong/mldds-courseware/raw/master/data/ucf101-5classes/train/Diving.zip
Extracted train/Diving
Downloaded https://github.com/lisaong/mldds-courseware/raw/master